<a href="https://colab.research.google.com/github/smirnovaanastasia1234/MOMO_II_Homework/blob/main/MOMO_II_Homework_II_Customer_Churn_Prediction_Part_2_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **МОМО II_Homework II_Customer Churn Prediction Optuna_Part 2**

Набор обучающих данных содержит 4250 выборок. Каждый образец содержит 19 функций и 1 логическую переменную «churn», которая указывает класс образца. 19 входных функций и 1 целевая переменная:


1.   "state" строка . 2-буквенный код штата США, где проживает клиент
2.   "account_length", numerical. Number of months the customer has been with the current telco provider
3. "area_code", string="area_code_AAA" where AAA = 3 digit area code
4. "international_plan", (yes/no). The customer has international plan.
5. "voice_mail_plan", (yes/no). The customer has voice mail plan.
6. "number_vmail_messages", numerical. Number of voice-mail messages.
7. "total_day_minutes", numerical. Total minutes of day calls.
8. "total_day_calls", numerical. Total minutes of day calls.
9. "total_day_charge", numerical. Total charge of day calls.
10. "total_eve_minutes", numerical. Total minutes of evening calls.
11. "total_eve_calls", numerical. Total number of evening calls.
12. "total_eve_charge", numerical. Total charge of evening calls.
13. "total_night_minutes", numerical. Total minutes of night calls.
14. "total_night_calls", numerical. Total number of night calls.
15. "total_night_charge", numerical. Total charge of night calls.
16. "total_intl_minutes", numerical. Total minutes of international calls.
17. "total_intl_calls", numerical. Total number of international calls.
18. "total_intl_charge", numerical. Total charge of international calls
19. "number_customer_service_calls", numerical. Number of calls to customer service
20. "churn", (yes/no). Customer churn - target variable.




# **Import библиотек**

In [62]:
import pandas as pd
import plotly as plt
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
from google.colab import files # чтобы загружать файлы в облако через проводник 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import preprocessing as prep 
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.pipeline import make_pipeline

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler, LabelEncoder
from sklearn.cluster import SpectralClustering

from sklearn.neighbors import  KNeighborsClassifier 
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import make_scorer 
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import ShuffleSplit 
from sklearn.model_selection import cross_val_score 
from sklearn import set_config
     

# **Загружаем Данные**

In [61]:
uploaded = files.upload() #просто запускаем и выбираем файл  API токена в проводнике

#↓ строка которая активирует наш API токен
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [63]:
!kaggle competitions download -c customer-churn-prediction-2020

customer-churn-prediction-2020.zip: Skipping, found more recently modified local copy (use --force to force download)


In [64]:
!unzip customer-churn-prediction-2020.zip

Archive:  customer-churn-prediction-2020.zip
replace sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: н
error:  invalid response [н]
replace sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: н
error:  invalid response [н]
replace sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sampleSubmission.csv    
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


# **Загружаем данные в Pandas**

In [65]:
df = pd.read_csv('train.csv')
df

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
0,OH,107,area_code_415,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,no
1,NJ,137,area_code_415,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,no
2,OH,84,area_code_408,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,no
3,OK,75,area_code_415,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,no
4,MA,121,area_code_510,no,yes,24,218.2,88,37.09,348.5,108,29.62,212.6,118,9.57,7.5,7,2.03,3,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4245,MT,83,area_code_415,no,no,0,188.3,70,32.01,243.8,88,20.72,213.7,79,9.62,10.3,6,2.78,0,no
4246,WV,73,area_code_408,no,no,0,177.9,89,30.24,131.2,82,11.15,186.2,89,8.38,11.5,6,3.11,3,no
4247,NC,75,area_code_408,no,no,0,170.7,101,29.02,193.1,126,16.41,129.1,104,5.81,6.9,7,1.86,1,no
4248,HI,50,area_code_408,no,yes,40,235.7,127,40.07,223.0,126,18.96,297.5,116,13.39,9.9,5,2.67,2,no


# Информация о DataFrame

In [66]:
df.info() # информация о данных в датафрейме

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4250 entries, 0 to 4249
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   state                          4250 non-null   object 
 1   account_length                 4250 non-null   int64  
 2   area_code                      4250 non-null   object 
 3   international_plan             4250 non-null   object 
 4   voice_mail_plan                4250 non-null   object 
 5   number_vmail_messages          4250 non-null   int64  
 6   total_day_minutes              4250 non-null   float64
 7   total_day_calls                4250 non-null   int64  
 8   total_day_charge               4250 non-null   float64
 9   total_eve_minutes              4250 non-null   float64
 10  total_eve_calls                4250 non-null   int64  
 11  total_eve_charge               4250 non-null   float64
 12  total_night_minutes            4250 non-null   f

# Поиск Дубликатов

In [67]:
df.duplicated().sum() ##Просуммируем и узнаем сколько всего дубликатов 

0

Дубликаты не обнаруженны

# EDA - Разведывательный анализ данных.

In [68]:
# определим категориальные и численные столбцы
cat_columns = []
num_columns = []

for column_name in df.columns:
    if (df[column_name].dtypes == object):
        cat_columns +=[column_name]
    else:
        num_columns +=[column_name]

print('Категориальные данные:\t ',cat_columns, '\n Число столблцов = ',len(cat_columns))

print('Числовые данные:\t ',  num_columns, '\n Число столблцов = ',len(num_columns))

Категориальные данные:	  ['state', 'area_code', 'international_plan', 'voice_mail_plan', 'churn'] 
 Число столблцов =  5
Числовые данные:	  ['account_length', 'number_vmail_messages', 'total_day_minutes', 'total_day_calls', 'total_day_charge', 'total_eve_minutes', 'total_eve_calls', 'total_eve_charge', 'total_night_minutes', 'total_night_calls', 'total_night_charge', 'total_intl_minutes', 'total_intl_calls', 'total_intl_charge', 'number_customer_service_calls'] 
 Число столблцов =  15


# Классификация Методом к-Ближайших Соседей

In [69]:
#Преобразуем целевую переменную
dct = {'no':0, 'yes':1}
df["churn"] = df["churn"].replace(dct)

df.groupby(df['churn'])['churn'].count().sort_values()
# разбиваем на тестовую и тренировочную
X_train, X_test, y_train, y_test = train_test_split(df.drop('churn', axis = 1), df['churn'], test_size=0.3, stratify = df['churn'], random_state=42)
     


In [70]:

cat_columns = X_train.select_dtypes(include = 'object').columns.tolist()
num_columns = X_train.select_dtypes(exclude = 'object').columns.tolist()

In [71]:
#Создаем конвейер для количественных переменных
num_pipe = Pipeline([('caler', StandardScaler())])

In [72]:
from numpy.lib.function_base import hamming
#Создаем конвейер для категориальных переменных
cat_pipe = Pipeline ([('ohe',OneHotEncoder(sparse=False, handle_unknown='ignore'))])

In [73]:
#Объединяем
transformers = [('num', num_pipe, num_columns),
                ('cat', cat_pipe, cat_columns)]

In [74]:
#передаем список трансформеров в класс
transformer = ColumnTransformer(transformers = transformers )

In [75]:
#смотрим, что получилось
transformer

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('caler', StandardScaler())]),
                                 ['account_length', 'number_vmail_messages',
                                  'total_day_minutes', 'total_day_calls',
                                  'total_day_charge', 'total_eve_minutes',
                                  'total_eve_calls', 'total_eve_charge',
                                  'total_night_minutes', 'total_night_calls',
                                  'total_night_charge', 'total_intl_minutes',
                                  'total_intl_calls', 'total_intl_charge',
                                  'number_customer_service_calls']),
                                ('cat',
                                 Pipeline(steps=[('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 ['state', 'area_code', 'international_plan',
                                  'voice_mail_plan'])])

In [76]:
#итоговый конвеер c knn
piple = Pipeline ([('transform', transformer),
                   ('knn', KNeighborsClassifier(n_neighbors=9, 
                           weights='distance'))])

In [77]:
#Обучаем
piple.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('caler',
                                                                   StandardScaler())]),
                                                  ['account_length',
                                                   'number_vmail_messages',
                                                   'total_day_minutes',
                                                   'total_day_calls',
                                                   'total_day_charge',
                                                   'total_eve_minutes',
                                                   'total_eve_calls',
                                                   'total_eve_charge',
                                                   'total_night_minutes',
                                                   'total_night_calls',
                                                   'total_night_charge',
                                                   'total_intl_minutes',
                                                   'total_intl_calls',
                                                   'total_intl_charge',
                                                   'number_customer_service_calls']),
                                                 ('cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['state', 'area_code',
                                                   'international_plan',
                                                   'voice_mail_plan'])])),
                ('knn',
                 KNeighborsClassifier(n_neighbors=9, weights='distance'))])

In [78]:
#Качество конвейера на тестовых данных
piple.score(X_test, y_test)

0.8901960784313725

# Поиск Лучших Гиперпараметров по Сетке Параметров

In [79]:
from sklearn.model_selection import GridSearchCV # для поиска по сетке гиперпараметров

In [85]:
parameters = {'knn__n_neighbors': [2, 5, 15, 30, 45, 64]}

In [86]:
kNN_search = GridSearchCV(piple, parameters, cv=10, scoring='accuracy')

In [87]:
kNN_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_outp

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('transform',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('caler',
                                                                                          StandardScaler())]),
                                                                         ['account_length',
                                                                          'number_vmail_messages',
                                                                          'total_day_minutes',
                                                                          'total_day_calls',
                                                                          'total_day_charge',
                                                                          'total_eve_minutes',
                                                                          'total_eve_calls',
                                                                          'total_eve_charge',
                                                                          'total_night_minutes',
                                                                          'total_night_calls',
                                                                          'total_nig...
                                                                          'total_intl_calls',
                                                                          'total_intl_charge',
                                                                          'number_customer_service_calls']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('ohe',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse=False))]),
                                                                         ['state',
                                                                          'area_code',
                                                                          'international_plan',
                                                                          'voice_mail_plan'])])),
                                       ('knn',
                                        KNeighborsClassifier(n_neighbors=9,
                                                             weights='distance'))]),
             param_grid={'knn__n_neighbors': [2, 5, 15, 30, 45, 64]},
             scoring='accuracy')

Лучший Кандидат

In [88]:
kNN_search.best_params_

{'knn__n_neighbors': 5}

In [89]:
#Проверим
piple_2 = Pipeline ([('transform', transformer),
                   ('knn', KNeighborsClassifier(n_neighbors=5, 
                           weights='distance'))])

In [90]:
#Обучаем
piple_2.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('caler',
                                                                   StandardScaler())]),
                                                  ['account_length',
                                                   'number_vmail_messages',
                                                   'total_day_minutes',
                                                   'total_day_calls',
                                                   'total_day_charge',
                                                   'total_eve_minutes',
                                                   'total_eve_calls',
                                                   'total_eve_charge',
                                                   'total_night_minutes',
                                                   'total_night_calls',
                                                   'total_night_charge',
                                                   'total_intl_minutes',
                                                   'total_intl_calls',
                                                   'total_intl_charge',
                                                   'number_customer_service_calls']),
                                                 ('cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['state', 'area_code',
                                                   'international_plan',
                                                   'voice_mail_plan'])])),
                ('knn', KNeighborsClassifier(weights='distance'))])

In [91]:
#Качество конвейера на тестовых данных
piple_2.score(X_test, y_test)

0.8933333333333333

Качество данных повысилось

# Поиск Лучших Гиперпараметров с Помощью Рандомизированного Поиска

In [94]:
RandomizedSearch = RandomizedSearchCV(piple, parameters, n_iter = 10, cv=5, random_state=42)

In [96]:
RandomizedSearch.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `spa

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('transform',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('caler',
                                                                                                StandardScaler())]),
                                                                               ['account_length',
                                                                                'number_vmail_messages',
                                                                                'total_day_minutes',
                                                                                'total_day_calls',
                                                                                'total_day_charge',
                                                                                'total_eve_minutes',
                                                                                'total_eve_calls',
                                                                                'total_eve_charge',
                                                                                'total_night_minutes',
                                                                                'total_night_calls',
                                                                                'tota...
                                                                                'total_intl_calls',
                                                                                'total_intl_charge',
                                                                                'number_customer_service_calls']),
                                                                              ('cat',
                                                                               Pipeline(steps=[('ohe',
                                                                                                OneHotEncoder(handle_unknown='ignore',
                                                                                                              sparse=False))]),
                                                                               ['state',
                                                                                'area_code',
                                                                                'international_plan',
                                                                                'voice_mail_plan'])])),
                                             ('knn',
                                              KNeighborsClassifier(n_neighbors=9,
                                                                   weights='distance'))]),
                   param_distributions={'knn__n_neighbors': [2, 5, 15, 30, 45,
                                                             64]},
                   random_state=42)

In [99]:
#запишем результаты поиска в DataFrame
result = pd.DataFrame(RandomizedSearch.cv_results_)
result

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_knn__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.034769,0.024808,0.027973,0.009648,2,{'knn__n_neighbors': 2},0.850420,0.830252,0.858824,0.875630,0.867227,0.856471,0.015571,6
1,0.017659,0.002283,0.027202,0.006226,5,{'knn__n_neighbors': 5},0.889076,0.880672,0.885714,0.889076,0.895798,0.888067,0.004940,1
2,0.016827,0.000624,0.028332,0.003398,15,{'knn__n_neighbors': 15},0.878992,0.880672,0.882353,0.878992,0.884034,0.881008,0.001960,2
3,0.017961,0.000318,0.031249,0.003032,30,{'knn__n_neighbors': 30},0.878992,0.875630,0.875630,0.880672,0.877311,0.877647,0.001960,3
4,0.018036,0.002450,0.039717,0.011012,45,{'knn__n_neighbors': 45},0.873950,0.870588,0.872269,0.877311,0.873950,0.873613,0.002230,4
5,0.018331,0.002060,0.041423,0.009511,64,{'knn__n_neighbors': 64},0.867227,0.870588,0.867227,0.872269,0.870588,0.869580,0.002017,5


In [102]:
DF_ = result[['param_knn__n_neighbors',
             'mean_test_score', 'std_test_score', 'rank_test_score']]
cm = sns.light_palette("green", as_cmap=True)
hl = DF_.sort_values(by = 'rank_test_score').style.background_gradient(cmap=cm)
hl
     

,param_knn__n_neighbors,mean_test_score,std_test_score,rank_test_score
1,5,0.888067,0.004940,1
2,15,0.881008,0.001960,2
3,30,0.877647,0.001960,3
4,45,0.873613,0.002230,4
5,64,0.869580,0.002017,5
0,2,0.856471,0.015571,6
